## <center> STA160 Final Exam

## <center> Web Scraping

## <center> 03 June 2022

In [1]:
# Import Libraries
import pandas as pd
import numpy as np

import requests
import json
import datetime
import itertools
from itertools import repeat

import matplotlib.pylab as plt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize'] = 10,6

## 1.1 Data for El Dorado County

In [2]:
dataE = pd.DataFrame()
new = []

date = ["03", "06", "07", "10"]

for d in date:
    start = "01-01-20" +d
    end = "12-31-20" +d
    dates = pd.date_range(start=start, end=end, freq = 'Y')
    dates = dates.strftime("%Y")   
    dates = dates.to_list()        
    
    Token = 'eLCmbnPGUBzvYdVYFMZYpulbSoXjYkfX'
    
    date_wind_speed = []
    date_rain = []
    date_tmax = []
    date_tmin = []
    
    wind_speed_lst = []
    rain_lst = []
    tmin_lst = []
    tmax_lst = []

    #query API for data of interest:
    for i in dates: #loop through list of years and append them to API call where appropriate
        temp_max = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMAX&limit=1000&locationid=FIPS:06017&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        rain = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&locationid=FIPS:06017&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        temp_min = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMIN&limit=1000&locationid=FIPS:06017&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        wind_speed = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=AWND&limit=1000&locationid=FIPS:06017&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        
        #convert to JSON:
        rain = rain.json()
        tmin = temp_min.json()
        tmax = temp_max.json()
        wind_speed = wind_speed.json()

        #create dataframe for each API call:
        rain_table = pd.json_normalize(rain["results"])
        tmin_table = pd.json_normalize(tmin["results"])
        tmax_table = pd.json_normalize(tmax["results"])
        wind_speed_table = pd.json_normalize(wind_speed["results"])

        #append data to appropriate list in order to construct overall dataset for El Dorado:
        date_rain.append(rain_table["date"].tolist())
        rain_lst.append(rain_table["value"].tolist())
        date_tmin.append(tmin_table['date'].tolist())
        tmin_lst.append(tmin_table['value'].tolist())
        date_tmax.append(tmax_table['date'].tolist())
        tmax_lst.append(tmax_table['value'].tolist())
        date_wind_speed.append(wind_speed_table["date"].tolist())
        wind_speed_lst.append(wind_speed_table["value"].tolist())

    #create dataset with date and value for each paramter of interest:
    rain_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_rain)),
        "Precip":list(itertools.chain.from_iterable(rain_lst))})

    tmax_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_tmax)),
        "tmax":list(itertools.chain.from_iterable(tmax_lst))})

    tmin_data = pd.DataFrame({
        "Date": list(itertools.chain.from_iterable(date_tmin)),
        "tmin":list(itertools.chain.from_iterable(tmin_lst))})
    
    wind_speed_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_wind_speed)),
        "wind_speed":list(itertools.chain.from_iterable(wind_speed_lst))})
        
    #set indices:
    rain_data = rain_data.set_index("Date")
    tmax_data = tmax_data.set_index("Date")
    tmin_data = tmin_data.set_index("Date")
    wind_speed_data = wind_speed_data.set_index("Date")
        
    #clean dates and set indices for merge:

    #clean date variable:
    rain_data.index = pd.to_datetime(rain_data.index)
    tmax_data.index = pd.to_datetime(tmax_data.index)
    tmin_data.index = pd.to_datetime(tmin_data.index)
    wind_speed_data.index = pd.to_datetime(wind_speed_data.index)

    #merge datasets into 1 dataset:
    eldo_dat = tmax_data.merge(tmin_data, how = "left",left_index=True, right_index=True)
    eldo_dat = eldo_dat.merge(rain_data, how = "inner",left_index=True, right_index=True)
    eldo_dat = eldo_dat.merge(wind_speed_data, how = "inner",left_index=True, right_index=True)
    eldo_dat["average"] = eldo_dat["tmin"]+((eldo_dat["tmax"]-eldo_dat["tmin"])/2) #compute average temperature
    eldo_dat['FIPS_CODE']= np.random.randint(17, len(eldo_dat))
    eldo_dat = eldo_dat.dropna()
    
    #append into one dataframe
    new.append(eldo_dat)

#show dataset:
dataE = pd.concat(new)
print(dataE)

            tmax  tmin  Precip  wind_speed  average  FIPS_CODE
Date                                                          
2003-01-01  50.0  42.0    0.00         2.7     46.0     128188
2003-01-01  50.0  42.0    0.01         2.7     46.0     128188
2003-01-01  50.0  42.0    0.09         2.7     46.0     128188
2003-01-01  50.0  42.0    0.00         2.7     46.0     128188
2003-01-01  50.0  42.0    0.20         2.7     46.0     128188
...          ...   ...     ...         ...      ...        ...
2010-03-04  36.0   4.0    0.79         3.1     20.0      83798
2010-03-04  36.0   4.0    0.00         3.1     20.0      83798
2010-03-04  36.0   4.0    0.00         3.1     20.0      83798
2010-03-04  36.0   4.0    0.00         3.1     20.0      83798
2010-03-04  36.0   4.0    0.00         3.1     20.0      83798

[603600 rows x 6 columns]


In [3]:
#save dataframe to csv
# dataE.to_csv("el_dorado_county.csv")

## 1.2 Data for Imperial County

In [4]:
dataI = pd.DataFrame()
new = []

date = ["03", "06", "07", "10"]

for d in date:
    start = "01-01-20" +d
    end = "12-31-20" +d
    dates = pd.date_range(start=start, end=end, freq = 'Y')
    dates = dates.strftime("%Y")   
    dates = dates.to_list()        
    
    Token = 'eLCmbnPGUBzvYdVYFMZYpulbSoXjYkfX'
    
    date_wind_speed = []
    date_rain = []
    date_tmax = []
    date_tmin = []
    
    wind_speed_lst = []
    rain_lst = []
    tmin_lst = []
    tmax_lst = []

    #query API for data of interest:
    for i in dates: #loop through list of years and append them to API call where appropriate
        temp_max = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMAX&limit=1000&locationid=FIPS:06025&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        rain = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&locationid=FIPS:06025&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        temp_min = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMIN&limit=1000&locationid=FIPS:06025&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        wind_speed = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=AWND&limit=1000&locationid=FIPS:06025&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        
        #convert to JSON:
        rain = rain.json()
        tmin = temp_min.json()
        tmax = temp_max.json()
        wind_speed = wind_speed.json()

        #create dataframe for each API call:
        rain_table = pd.json_normalize(rain["results"])
        tmin_table = pd.json_normalize(tmin["results"])
        tmax_table = pd.json_normalize(tmax["results"])
        wind_speed_table = pd.json_normalize(wind_speed["results"])

        #append data to appropriate list in order to construct overall dataset for Imperial:
        date_rain.append(rain_table["date"].tolist())
        rain_lst.append(rain_table["value"].tolist())
        date_tmin.append(tmin_table['date'].tolist())
        tmin_lst.append(tmin_table['value'].tolist())
        date_tmax.append(tmax_table['date'].tolist())
        tmax_lst.append(tmax_table['value'].tolist())
        date_wind_speed.append(wind_speed_table["date"].tolist())
        wind_speed_lst.append(wind_speed_table["value"].tolist())

    #create dataset with date and value for each paramter of interest:
    rain_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_rain)),
        "Precip":list(itertools.chain.from_iterable(rain_lst))})

    tmax_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_tmax)),
        "tmax":list(itertools.chain.from_iterable(tmax_lst))})

    tmin_data = pd.DataFrame({
        "Date": list(itertools.chain.from_iterable(date_tmin)),
        "tmin":list(itertools.chain.from_iterable(tmin_lst))})
    
    wind_speed_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_wind_speed)),
        "wind_speed":list(itertools.chain.from_iterable(wind_speed_lst))})
        
    #set indices:
    rain_data = rain_data.set_index("Date")
    tmax_data = tmax_data.set_index("Date")
    tmin_data = tmin_data.set_index("Date")
    wind_speed_data = wind_speed_data.set_index("Date")
        
    #clean dates and set indices for merge:

    #clean date variable:
    rain_data.index = pd.to_datetime(rain_data.index)
    tmax_data.index = pd.to_datetime(tmax_data.index)
    tmin_data.index = pd.to_datetime(tmin_data.index)
    wind_speed_data.index = pd.to_datetime(wind_speed_data.index)

    #merge datasets into 1 dataset:
    imperial_dat = tmax_data.merge(tmin_data, how = "left",left_index=True, right_index=True)
    imperial_dat = imperial_dat.merge(rain_data, how = "inner",left_index=True, right_index=True)
    imperial_dat = imperial_dat.merge(wind_speed_data, how = "inner",left_index=True, right_index=True)
    imperial_dat["average"] = imperial_dat["tmin"]+((imperial_dat["tmax"]-imperial_dat["tmin"])/2) #compute average temperature
    imperial_dat = imperial_dat.dropna()
    
    #append into one dataframe
    new.append(imperial_dat)

#show dataset:
dataI = pd.concat(new)
print(dataI)

            tmax  tmin  Precip  wind_speed  average
Date                                               
2003-01-01  70.0  37.0    0.00         5.6     53.5
2003-01-01  70.0  37.0    0.00         5.6     53.5
2003-01-01  70.0  37.0    0.00         5.6     53.5
2003-01-01  70.0  37.0    0.00         5.6     53.5
2003-01-01  70.0  37.0    0.00         5.6     53.5
...          ...   ...     ...         ...      ...
2010-04-22  67.0  48.0    0.00        13.4     57.5
2010-04-22  67.0  48.0    0.02        11.2     57.5
2010-04-22  67.0  48.0    0.02        13.4     57.5
2010-04-22  67.0  48.0    0.01        11.2     57.5
2010-04-22  67.0  48.0    0.01        13.4     57.5

[386155 rows x 5 columns]


In [5]:
#save to csv:
# dataI.to_csv("Imperial_county.csv")

## 1.3 Data for Mendocino County

In [6]:
dataM = pd.DataFrame()
new = []

date = ["03", "06", "07", "10"]

for d in date:
    start = "01-01-20" +d
    end = "12-31-20" +d
    dates = pd.date_range(start=start, end=end, freq = 'Y')
    dates = dates.strftime("%Y")   
    dates = dates.to_list()        
    
    Token = 'eLCmbnPGUBzvYdVYFMZYpulbSoXjYkfX'
    
    date_wind_speed = []
    date_rain = []
    date_tmax = []
    date_tmin = []
    
    wind_speed_lst = []
    rain_lst = []
    tmin_lst = []
    tmax_lst = []

    #query API for data of interest:
    for i in dates: #loop through list of years and append them to API call where appropriate
        temp_max = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMAX&limit=1000&locationid=FIPS:06045&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        rain = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&locationid=FIPS:06045&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        temp_min = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMIN&limit=1000&locationid=FIPS:06045&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        wind_speed = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=AWND&limit=1000&locationid=FIPS:06045&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        
        #convert to JSON:
        rain = rain.json()
        tmin = temp_min.json()
        tmax = temp_max.json()
        wind_speed = wind_speed.json()

        #create dataframe for each API call:
        rain_table = pd.json_normalize(rain["results"])
        tmin_table = pd.json_normalize(tmin["results"])
        tmax_table = pd.json_normalize(tmax["results"])
        wind_speed_table = pd.json_normalize(wind_speed["results"])

        #append data to appropriate list in order to construct overall dataset for Mendocino:
        date_rain.append(rain_table["date"].tolist())
        rain_lst.append(rain_table["value"].tolist())
        date_tmin.append(tmin_table['date'].tolist())
        tmin_lst.append(tmin_table['value'].tolist())
        date_tmax.append(tmax_table['date'].tolist())
        tmax_lst.append(tmax_table['value'].tolist())
        date_wind_speed.append(wind_speed_table["date"].tolist())
        wind_speed_lst.append(wind_speed_table["value"].tolist())

    #create dataset with date and value for each paramter of interest:
    rain_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_rain)),
        "Precip":list(itertools.chain.from_iterable(rain_lst))})

    tmax_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_tmax)),
        "tmax":list(itertools.chain.from_iterable(tmax_lst))})

    tmin_data = pd.DataFrame({
        "Date": list(itertools.chain.from_iterable(date_tmin)),
        "tmin":list(itertools.chain.from_iterable(tmin_lst))})
    
    wind_speed_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_wind_speed)),
        "wind_speed":list(itertools.chain.from_iterable(wind_speed_lst))})
        
    #set indices:
    rain_data = rain_data.set_index("Date")
    tmax_data = tmax_data.set_index("Date")
    tmin_data = tmin_data.set_index("Date")
    wind_speed_data = wind_speed_data.set_index("Date")
        
    #clean dates and set indices for merge:

    #clean date variable:
    rain_data.index = pd.to_datetime(rain_data.index)
    tmax_data.index = pd.to_datetime(tmax_data.index)
    tmin_data.index = pd.to_datetime(tmin_data.index)
    wind_speed_data.index = pd.to_datetime(wind_speed_data.index)

    #merge datasets into 1 dataset:
    Mendocino_dat = tmax_data.merge(tmin_data, how = "left",left_index=True, right_index=True)
    Mendocino_dat = imperial_dat.merge(rain_data, how = "inner",left_index=True, right_index=True)
    Mendocino_dat = imperial_dat.merge(wind_speed_data, how = "inner",left_index=True, right_index=True)
    Mendocino_dat["average"] = Mendocino_dat["tmin"]+((Mendocino_dat["tmax"]-Mendocino_dat["tmin"])/2) #compute average temperature
    Mendocino_dat = Mendocino_dat.dropna()
    
    #append into one dataframe
    new.append(Mendocino_dat)

#show dataset:
dataM = pd.concat(new)
print(dataM)

            tmax  tmin  Precip  wind_speed_x  average  wind_speed_y
Date                                                               
2010-01-01  69.0  41.0    0.00           4.3     55.0           5.4
2010-01-01  69.0  41.0    0.00           4.9     55.0           5.4
2010-01-01  69.0  41.0    0.00           4.3     55.0           5.4
2010-01-01  69.0  41.0    0.00           4.9     55.0           5.4
2010-01-01  69.0  41.0    0.00           4.3     55.0           5.4
...          ...   ...     ...           ...      ...           ...
2010-04-22  67.0  48.0    0.00          13.4     57.5           2.9
2010-04-22  67.0  48.0    0.02          11.2     57.5           2.9
2010-04-22  67.0  48.0    0.02          13.4     57.5           2.9
2010-04-22  67.0  48.0    0.01          11.2     57.5           2.9
2010-04-22  67.0  48.0    0.01          13.4     57.5           2.9

[113732 rows x 6 columns]


In [7]:
#save to csv:
dataM.to_csv("Mendocino_county.csv")

## 1.4 Data for Riverside

In [8]:
dataR = pd.DataFrame()
new = []

date = ["03", "06", "07", "10"]

for d in date:
    start = "01-01-20" +d
    end = "12-31-20" +d
    dates = pd.date_range(start=start, end=end, freq = 'Y')
    dates = dates.strftime("%Y")   
    dates = dates.to_list()        
    
    Token = 'eLCmbnPGUBzvYdVYFMZYpulbSoXjYkfX'
    
    date_wind_speed = []
    date_rain = []
    date_tmax = []
    date_tmin = []
    
    wind_speed_lst = []
    rain_lst = []
    tmin_lst = []
    tmax_lst = []

    #query API for data of interest:
    for i in dates: #loop through list of years and append them to API call where appropriate
        temp_max = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMAX&limit=1000&locationid=FIPS:06045&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        rain = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&locationid=FIPS:06045&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        temp_min = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMIN&limit=1000&locationid=FIPS:06045&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        wind_speed = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=AWND&limit=1000&locationid=FIPS:06045&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        
        #convert to JSON:
        rain = rain.json()
        tmin = temp_min.json()
        tmax = temp_max.json()
        wind_speed = wind_speed.json()

        #create dataframe for each API call:
        rain_table = pd.json_normalize(rain["results"])
        tmin_table = pd.json_normalize(tmin["results"])
        tmax_table = pd.json_normalize(tmax["results"])
        wind_speed_table = pd.json_normalize(wind_speed["results"])

        #append data to appropriate list in order to construct overall dataset for Riverside:
        date_rain.append(rain_table["date"].tolist())
        rain_lst.append(rain_table["value"].tolist())
        date_tmin.append(tmin_table['date'].tolist())
        tmin_lst.append(tmin_table['value'].tolist())
        date_tmax.append(tmax_table['date'].tolist())
        tmax_lst.append(tmax_table['value'].tolist())
        date_wind_speed.append(wind_speed_table["date"].tolist())
        wind_speed_lst.append(wind_speed_table["value"].tolist())

    #create dataset with date and value for each paramter of interest:
    rain_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_rain)),
        "Precip":list(itertools.chain.from_iterable(rain_lst))})

    tmax_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_tmax)),
        "tmax":list(itertools.chain.from_iterable(tmax_lst))})

    tmin_data = pd.DataFrame({
        "Date": list(itertools.chain.from_iterable(date_tmin)),
        "tmin":list(itertools.chain.from_iterable(tmin_lst))})
    
    wind_speed_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_wind_speed)),
        "wind_speed":list(itertools.chain.from_iterable(wind_speed_lst))})
        
    #set indices:
    rain_data = rain_data.set_index("Date")
    tmax_data = tmax_data.set_index("Date")
    tmin_data = tmin_data.set_index("Date")
    wind_speed_data = wind_speed_data.set_index("Date")
        
    #clean dates and set indices for merge:

    #clean date variable:
    rain_data.index = pd.to_datetime(rain_data.index)
    tmax_data.index = pd.to_datetime(tmax_data.index)
    tmin_data.index = pd.to_datetime(tmin_data.index)
    wind_speed_data.index = pd.to_datetime(wind_speed_data.index)

    #merge datasets into 1 dataset:
    Riverside_dat = tmax_data.merge(tmin_data, how = "left",left_index=True, right_index=True)
    Riverside_dat = Riverside_dat.merge(rain_data, how = "inner",left_index=True, right_index=True)
    Riverside_dat = Riverside_dat.merge(wind_speed_data, how = "inner",left_index=True, right_index=True)
    Riverside_dat["average"] = Riverside_dat["tmin"]+((Riverside_dat["tmax"]-Riverside_dat["tmin"])/2) #compute average temperature
    Riverside_dat = Riverside_dat.dropna()
    
    #append into one dataframe
    new.append(Riverside_dat)

#show dataset:
dataR = pd.concat(new)
print(dataR)

            tmax  tmin  Precip  wind_speed  average
Date                                               
2003-01-01  46.0  32.0    0.00         0.7     39.0
2003-01-01  46.0  32.0    0.00         0.7     39.0
2003-01-01  46.0  32.0    0.04         0.7     39.0
2003-01-01  46.0  32.0    0.00         0.7     39.0
2003-01-01  46.0  32.0    0.00         0.7     39.0
...          ...   ...     ...         ...      ...
2010-02-25  59.0  36.0    0.00         2.5     47.5
2010-02-25  59.0  36.0    0.01         2.5     47.5
2010-02-25  59.0  36.0    0.00         2.5     47.5
2010-02-25  59.0  36.0    0.00         2.5     47.5
2010-02-25  59.0  36.0    0.09         2.5     47.5

[475420 rows x 5 columns]


In [9]:
#save to csv:
dataR.to_csv("Riverside_county.csv")

## 1.5 Data for Siskiyou County

In [10]:
dataS = pd.DataFrame()
new = []

date = ["03", "06", "07", "10"]

for d in date:
    start = "01-01-20" +d
    end = "12-31-20" +d
    dates = pd.date_range(start=start, end=end, freq = 'Y')
    dates = dates.strftime("%Y")   
    dates = dates.to_list()        
    
    Token = 'eLCmbnPGUBzvYdVYFMZYpulbSoXjYkfX'
    
    date_wind_speed = []
    date_rain = []
    date_tmax = []
    date_tmin = []
    
    wind_speed_lst = []
    rain_lst = []
    tmin_lst = []
    tmax_lst = []

    #query API for data of interest:
    for i in dates: #loop through list of years and append them to API call where appropriate
        temp_max = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMAX&limit=1000&locationid=FIPS:06093&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        rain = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&locationid=FIPS:06093&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        temp_min = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMIN&limit=1000&locationid=FIPS:06093&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        wind_speed = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=AWND&limit=1000&locationid=FIPS:06093&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        
        #convert to JSON:
        rain = rain.json()
        tmin = temp_min.json()
        tmax = temp_max.json()
        wind_speed = wind_speed.json()

        #create dataframe for each API call:
        rain_table = pd.json_normalize(rain["results"])
        tmin_table = pd.json_normalize(tmin["results"])
        tmax_table = pd.json_normalize(tmax["results"])
        wind_speed_table = pd.json_normalize(wind_speed["results"])

        #append data to appropriate list in order to construct overall dataset for Siskiyou:
        date_rain.append(rain_table["date"].tolist())
        rain_lst.append(rain_table["value"].tolist())
        date_tmin.append(tmin_table['date'].tolist())
        tmin_lst.append(tmin_table['value'].tolist())
        date_tmax.append(tmax_table['date'].tolist())
        tmax_lst.append(tmax_table['value'].tolist())
        date_wind_speed.append(wind_speed_table["date"].tolist())
        wind_speed_lst.append(wind_speed_table["value"].tolist())

    #create dataset with date and value for each paramter of interest:
    rain_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_rain)),
        "Precip":list(itertools.chain.from_iterable(rain_lst))})

    tmax_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_tmax)),
        "tmax":list(itertools.chain.from_iterable(tmax_lst))})

    tmin_data = pd.DataFrame({
        "Date": list(itertools.chain.from_iterable(date_tmin)),
        "tmin":list(itertools.chain.from_iterable(tmin_lst))})
    
    wind_speed_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_wind_speed)),
        "wind_speed":list(itertools.chain.from_iterable(wind_speed_lst))})
        
    #set indices:
    rain_data = rain_data.set_index("Date")
    tmax_data = tmax_data.set_index("Date")
    tmin_data = tmin_data.set_index("Date")
    wind_speed_data = wind_speed_data.set_index("Date")
        
    #clean dates and set indices for merge:

    #clean date variable:
    rain_data.index = pd.to_datetime(rain_data.index)
    tmax_data.index = pd.to_datetime(tmax_data.index)
    tmin_data.index = pd.to_datetime(tmin_data.index)
    wind_speed_data.index = pd.to_datetime(wind_speed_data.index)

    #merge datasets into 1 dataset:
    Sisk_dat = tmax_data.merge(tmin_data, how = "left",left_index=True, right_index=True)
    Sisk_dat = Sisk_dat.merge(rain_data, how = "inner",left_index=True, right_index=True)
    Sisk_dat = Sisk_dat.merge(wind_speed_data, how = "inner",left_index=True, right_index=True)
    Sisk_dat["average"] = Sisk_dat["tmin"]+((Sisk_dat["tmax"]-Sisk_dat["tmin"])/2) #compute average temperature
    Sisk_dat = Sisk_dat.dropna()
    
    #append into one dataframe
    new.append(Sisk_dat)

#show dataset:
dataS = pd.concat(new)
print(dataS)

            tmax  tmin  Precip  wind_speed  average
Date                                               
2003-01-01  45.0  28.0    0.02         0.7     36.5
2003-01-01  45.0  28.0    0.02         4.5     36.5
2003-01-01  45.0  28.0    0.05         0.7     36.5
2003-01-01  45.0  28.0    0.05         4.5     36.5
2003-01-01  45.0  28.0    0.02         0.7     36.5
...          ...   ...     ...         ...      ...
2010-02-09  44.0  29.0    0.04         1.8     36.5
2010-02-09  44.0  29.0    0.07         0.9     36.5
2010-02-09  44.0  29.0    0.07         1.8     36.5
2010-02-09  44.0  29.0    0.03         0.9     36.5
2010-02-09  44.0  29.0    0.03         1.8     36.5

[2602773 rows x 5 columns]


In [11]:
#save to csv:
dataS.to_csv("Siskyou_county.csv")

## 1.6 Data for Solano County

In [12]:
dataL = pd.DataFrame()
new = []

date = ["03", "06", "07", "10"]

for d in date:
    start = "01-01-20" +d
    end = "12-31-20" +d
    dates = pd.date_range(start=start, end=end, freq = 'Y')
    dates = dates.strftime("%Y")   
    dates = dates.to_list()        
    
    Token = 'eLCmbnPGUBzvYdVYFMZYpulbSoXjYkfX'
    
#     date_wind_speed = []
    date_rain = []
    date_tmax = []
    date_tmin = []
    
#     wind_speed_lst = []
    rain_lst = []
    tmin_lst = []
    tmax_lst = []

    #query API for data of interest:
    for i in dates: #loop through list of years and append them to API call where appropriate
        temp_max = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMAX&limit=1000&locationid=FIPS:06095&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        rain = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&locationid=FIPS:06095&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        temp_min = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMIN&limit=1000&locationid=FIPS:06095&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
#         wind_speed = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=AWND&limit=1000&locationid=FIPS:06095&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        
        #convert to JSON:
        rain = rain.json()
        tmin = temp_min.json()
        tmax = temp_max.json()
#         wind_speed = wind_speed.json()

        #create dataframe for each API call:
        rain_table = pd.json_normalize(rain["results"])
        tmin_table = pd.json_normalize(tmin["results"])
        tmax_table = pd.json_normalize(tmax["results"])
#         wind_speed_table = pd.json_normalize(wind_speed["results"])

        #append data to appropriate list in order to construct overall dataset for Solano:
        date_rain.append(rain_table["date"].tolist())
        rain_lst.append(rain_table["value"].tolist())
        date_tmin.append(tmin_table['date'].tolist())
        tmin_lst.append(tmin_table['value'].tolist())
        date_tmax.append(tmax_table['date'].tolist())
        tmax_lst.append(tmax_table['value'].tolist())
#         date_wind_speed.append(wind_speed_table["date"].tolist())
#         wind_speed_lst.append(wind_speed_table["value"].tolist())

    #create dataset with date and value for each paramter of interest:
    rain_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_rain)),
        "Precip":list(itertools.chain.from_iterable(rain_lst))})

    tmax_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_tmax)),
        "tmax":list(itertools.chain.from_iterable(tmax_lst))})

    tmin_data = pd.DataFrame({
        "Date": list(itertools.chain.from_iterable(date_tmin)),
        "tmin":list(itertools.chain.from_iterable(tmin_lst))})
    
#     wind_speed_data = pd.DataFrame({
#         "Date":list(itertools.chain.from_iterable(date_wind_speed)),
#         "wind_speed":list(itertools.chain.from_iterable(wind_speed_lst))})
        
    #set indices:
    rain_data = rain_data.set_index("Date")
    tmax_data = tmax_data.set_index("Date")
    tmin_data = tmin_data.set_index("Date")
#     wind_speed_data = wind_speed_data.set_index("Date")
        
    #clean dates and set indices for merge:

    #clean date variable:
    rain_data.index = pd.to_datetime(rain_data.index)
    tmax_data.index = pd.to_datetime(tmax_data.index)
    tmin_data.index = pd.to_datetime(tmin_data.index)
#     wind_speed_data.index = pd.to_datetime(wind_speed_data.index)

    #merge datasets into 1 dataset:
    solano_dat = tmax_data.merge(tmin_data, how = "left",left_index=True, right_index=True)
    solano_dat = solano_dat.merge(rain_data, how = "inner",left_index=True, right_index=True)
#     solano_dat = solano_dat.merge(wind_speed_data, how = "inner",left_index=True, right_index=True)
    solano_dat["average"] = solano_dat["tmin"]+((solano_dat["tmax"]-solano_dat["tmin"])/2) #compute average temperature
    solano_dat = solano_dat.dropna()
    
    #append into one dataframe
    new.append(solano_dat)

#show dataset:
dataL = pd.concat(new)
print(dataL)

            tmax  tmin  Precip  average
Date                                   
2003-01-01  56.0  37.0    0.00     46.5
2003-01-01  56.0  37.0    0.00     46.5
2003-01-01  56.0  37.0    0.00     46.5
2003-01-01  56.0  37.0    0.00     46.5
2003-01-01  56.0  37.0    0.01     46.5
...          ...   ...     ...      ...
2010-09-01  99.0  65.0    0.00     82.0
2010-09-01  99.0  60.0    0.00     79.5
2010-09-01  99.0  60.0    0.00     79.5
2010-09-01  99.0  60.0    0.00     79.5
2010-09-01  99.0  60.0    0.00     79.5

[105508 rows x 4 columns]


In [13]:
#save to csv:
dataL.to_csv("Solano_county.csv")

## 1.7 Data for Merced County

In [14]:
dataMC = pd.DataFrame()
new = []

date = ["03", "06", "07", "10"]

for d in date:
    start = "01-01-20" +d
    end = "12-31-20" +d
    dates = pd.date_range(start=start, end=end, freq = 'Y')
    dates = dates.strftime("%Y")   
    dates = dates.to_list()        
    
    Token = 'eLCmbnPGUBzvYdVYFMZYpulbSoXjYkfX'
    
    date_wind_speed = []
    date_rain = []
    date_tmax = []
    date_tmin = []
    
    wind_speed_lst = []
    rain_lst = []
    tmin_lst = []
    tmax_lst = []

    #query API for data of interest:
    for i in dates: #loop through list of years and append them to API call where appropriate
        temp_max = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMAX&limit=1000&locationid=FIPS:06047&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        rain = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=PRCP&limit=1000&locationid=FIPS:06047&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        temp_min = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TMIN&limit=1000&locationid=FIPS:06047&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        wind_speed = requests.get("https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=AWND&limit=1000&locationid=FIPS:06047&units=standard&startdate="+str(i)+"-01-01&enddate="+str(i)+"-12-31", headers={'token':Token})
        
        #convert to JSON:
        rain = rain.json()
        tmin = temp_min.json()
        tmax = temp_max.json()
        wind_speed = wind_speed.json()

        #create dataframe for each API call:
        rain_table = pd.json_normalize(rain["results"])
        tmin_table = pd.json_normalize(tmin["results"])
        tmax_table = pd.json_normalize(tmax["results"])
        wind_speed_table = pd.json_normalize(wind_speed["results"])

        #append data to appropriate list in order to construct overall dataset for Merced:
        date_rain.append(rain_table["date"].tolist())
        rain_lst.append(rain_table["value"].tolist())
        date_tmin.append(tmin_table['date'].tolist())
        tmin_lst.append(tmin_table['value'].tolist())
        date_tmax.append(tmax_table['date'].tolist())
        tmax_lst.append(tmax_table['value'].tolist())
        date_wind_speed.append(wind_speed_table["date"].tolist())
        wind_speed_lst.append(wind_speed_table["value"].tolist())

    #create dataset with date and value for each paramter of interest:
    rain_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_rain)),
        "Precip":list(itertools.chain.from_iterable(rain_lst))})

    tmax_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_tmax)),
        "tmax":list(itertools.chain.from_iterable(tmax_lst))})

    tmin_data = pd.DataFrame({
        "Date": list(itertools.chain.from_iterable(date_tmin)),
        "tmin":list(itertools.chain.from_iterable(tmin_lst))})
    
    wind_speed_data = pd.DataFrame({
        "Date":list(itertools.chain.from_iterable(date_wind_speed)),
        "wind_speed":list(itertools.chain.from_iterable(wind_speed_lst))})
        
    #set indices:
    rain_data = rain_data.set_index("Date")
    tmax_data = tmax_data.set_index("Date")
    tmin_data = tmin_data.set_index("Date")
    wind_speed_data = wind_speed_data.set_index("Date")
        
    #clean dates and set indices for merge:

    #clean date variable:
    rain_data.index = pd.to_datetime(rain_data.index)
    tmax_data.index = pd.to_datetime(tmax_data.index)
    tmin_data.index = pd.to_datetime(tmin_data.index)
    wind_speed_data.index = pd.to_datetime(wind_speed_data.index)

    #merge datasets into 1 dataset:
    merced_dat = tmax_data.merge(tmin_data, how = "left",left_index=True, right_index=True)
    merced_dat = merced_dat.merge(rain_data, how = "inner",left_index=True, right_index=True)
    merced_dat = merced_dat.merge(wind_speed_data, how = "inner",left_index=True, right_index=True)
    merced_dat["average"] = merced_dat["tmin"]+((merced_dat["tmax"]-merced_dat["tmin"])/2) #compute average temperature
    merced_dat = merced_dat.dropna()
    
    #append into one dataframe
    new.append(merced_dat)

#show dataset:
dataMC = pd.concat(new)
print(dataMC)

            tmax  tmin  Precip  wind_speed  average
Date                                               
2003-01-01  57.0  34.0    0.00         1.8     45.5
2003-01-01  57.0  34.0    0.02         1.8     45.5
2003-01-01  57.0  34.0    0.00         1.8     45.5
2003-01-01  57.0  34.0    0.03         1.8     45.5
2003-01-01  57.0  34.0    0.00         1.8     45.5
...          ...   ...     ...         ...      ...
2010-05-25  75.0  43.0    0.01         4.9     59.0
2010-05-25  75.0  43.0    0.00         4.9     59.0
2010-05-25  75.0  43.0    0.00         4.9     59.0
2010-05-25  75.0  43.0    0.13         4.9     59.0
2010-05-25  75.0  43.0    0.12         4.9     59.0

[231772 rows x 5 columns]


In [15]:
#save to csv:
# dataMC.to_csv("Merced_county.csv")

## Combined Fire and Weather Dataset

In [16]:
df = pd.read_csv("/Users/selamawit/Desktop/STA160/Final/data/all_com_wind.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12183 entries, 0 to 12182
Data columns (total 23 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        12183 non-null  int64  
 1   Date              12183 non-null  object 
 2   wind_speed        12183 non-null  float64
 3   FIPS_NAME_x       12183 non-null  object 
 4   FPA_ID            315 non-null    object 
 5   FIRE_CODE         315 non-null    object 
 6   FIRE_NAME         315 non-null    object 
 7   FIRE_YEAR         315 non-null    float64
 8   DISCOVERY_TIME    315 non-null    float64
 9   STAT_CAUSE_CODE   315 non-null    float64
 10  STAT_CAUSE_DESCR  315 non-null    object 
 11  date(CONT_DATE)   315 non-null    object 
 12  FIRE_SIZE         315 non-null    float64
 13  FIRE_SIZE_CLASS   315 non-null    object 
 14  LATITUDE          315 non-null    float64
 15  LONGITUDE         315 non-null    float64
 16  FIPS_CODE         315 non-null    float6